<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [1]:
# Install PySpark.

In [2]:
!pip install pyspark==3.3.2
!pip install findspark
!pip install pandas

In [3]:
# Import Libraries.

In [4]:
import findspark
findspark.init()

In [5]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd

In [6]:
# Start a Session

In [7]:
# Creating a Spark Session and Context.
spark = SparkSession \
    .builder \
    .appName("SparkML Ops").getOrCreate()

23/03/27 05:42:14 WARN Utils: Your hostname, hp-envy15 resolves to a loopback address: 127.0.1.1; using 192.168.2.32 instead (on interface wlxd03745a459a2)
23/03/27 05:42:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/27 05:42:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [9]:
!wget -N "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv"

--2023-03-27 05:42:17--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘searchterms.csv’ not modified on server. Omitting download.



In [10]:
# Load the csv into a spark dataframe

In [11]:
searchterms = pd.read_csv('searchterms.csv')
sdf = spark.createDataFrame(searchterms)

In [12]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [13]:
row_count = sdf.count()
col_count = len(sdf.columns)
print(f'Total number of rows: {row_count}')
print(f'Total number of columns: {col_count}')

Total number of rows: 10000
Total number of columns: 4


In [14]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [15]:
print(sdf.show(5))

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows

None


In [16]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [17]:
sdf.printSchema()

root
 |-- day: long (nullable = true)
 |-- month: long (nullable = true)
 |-- year: long (nullable = true)
 |-- searchterm: string (nullable = true)



In [18]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [19]:
sdf.createOrReplaceTempView("sdf")
spark.sql("""select count(*) as gaming_laptop from sdf where searchterm='gaming laptop'""").show()

+-------------+
|gaming_laptop|
+-------------+
|          499|
+-------------+



In [20]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [21]:
spark.sql("""select count(*), searchterm as gaming_laptop from sdf group by searchterm order by count(*) desc""").show(5)

+--------+-------------+
|count(1)|gaming_laptop|
+--------+-------------+
|    2312|mobile 6 inch|
|    2301|    mobile 5g|
|    1327|mobile latest|
|     935|       laptop|
|     896|  tablet wifi|
+--------+-------------+
only showing top 5 rows



In [22]:
# Import SparkML libraries

In [23]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [24]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [25]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [26]:
!wget -N "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz"
#!gzip -dk model.gzip
!tar -xvzf model.tar.gz

--2023-03-27 05:42:27--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2023-03-27 05:42:28 (340 MB/s) - ‘model.tar.gz’ saved [1490/1490]

sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_prediction.model/data/
sale

In [27]:
# You need LinearRegressionModel to load the model
from pyspark.ml.regression import LinearRegressionModel
model = LinearRegressionModel.load('sales_prediction.model')

In [28]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [29]:
def predict(year):
    assembler = VectorAssembler(inputCols=["year"],outputCol="features")
    data = [[year,0]]
    columns = ["year", "sales"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features', 'year')
    predictions = model.transform(__)
    predictions.select('prediction').show()

predict(2023)

+------------------+
|        prediction|
+------------------+
|176.14285712605306|
+------------------+

